In [1]:
import pandas as pd

In [1]:
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

from scr.bubble import *

In [ ]:
df = get("Feedback")

In [3]:
#df = pd.read_csv('Insights.csv')
df.columns = ['Insight', 'InsightID', 'InsightParentID', 'IsParent']
#df.head()

In [4]:
filter_no_parents = df['InsightParentID'] != df['InsightParentID']
fltered_df = df[filter_no_parents]
fltered_df['InsightParentID'] = fltered_df['InsightID']
df[filter_no_parents] = fltered_df


In [5]:
df["InsightParent"] = df["InsightParentID"].apply(lambda x: df.iloc[df['InsightID'].eq(x).idxmax()]["Insight"])

In [6]:
df.describe()

,Insight,InsightID,InsightParentID,IsParent,InsightParent
count,297,297,297,297,297
unique,297,297,70,2,70
top,"Les intérimaires manquent de rigueur en termes de ponctualité, de professionnalisme, et d'attention au détail, ce qui se traduit par des erreurs coûteuses et un comportement inapproprié au travail.",1702251558470x613868062724425100,1702251795709x628710337044019200,non,"Les clients apprécient grandement le professionnalisme, la qualité d'écoute et l'excellence des interactions avec la conseillère de Randstad, reflétant un service client hautement satisfaisant."
freq,1,1,33,254,33


In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('OrdalieTech/Solon-embeddings-large-0.1')
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

No sentence-transformers model found with name /Users/gardille/.cache/torch/sentence_transformers/lxyuan_distilbert-base-multilingual-cased-sentiments-student. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /Users/gardille/.cache/torch/sentence_transformers/lxyuan_distilbert-base-multilingual-cased-sentiments-student were not used when initializing DistilBertModel: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
sentences = df['Insight']
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(297, 1024)

In [9]:
sentence_sentiments = sentiment_model.encode(sentences)
sentence_sentiments

array([[-0.6170026 ,  0.33922127, -0.16932467, ..., -0.19149339,
        -0.33608186, -0.18402731],
       [-0.30589217,  0.29113886, -0.06420135, ...,  0.05913984,
        -0.02826975,  0.01194301],
       [ 0.03208002,  0.1750952 , -0.05016001, ...,  0.09668454,
         0.10843945,  0.06135543],
       ...,
       [-0.02473159, -0.1065196 , -0.25289848, ...,  0.03104051,
         0.12627216,  0.01539773],
       [ 0.02705233,  0.04989604, -0.2614626 , ..., -0.06167848,
        -0.02925681, -0.01573835],
       [-0.05509756,  0.02750333, -0.20200625, ..., -0.18028787,
         0.0190844 , -0.06038549]], dtype=float32)

In [10]:
sentence_embeddings

array([[-0.04266007, -0.01242622,  0.03966972, ..., -0.02307059,
        -0.09413584, -0.03576062],
       [-0.01890397, -0.02291213, -0.00415571, ..., -0.00181556,
        -0.01434652, -0.0462789 ],
       [-0.06531541, -0.00406456, -0.00932761, ..., -0.04511409,
        -0.03815044, -0.04094359],
       ...,
       [-0.03131469,  0.02609637,  0.02687677, ..., -0.02091159,
        -0.06056459, -0.03400551],
       [ 0.00440707, -0.04418156,  0.00804911, ..., -0.02692328,
        -0.07442538, -0.01602571],
       [-0.05931276, -0.0298768 ,  0.00861773, ..., -0.06241287,
        -0.02789943, -0.03819002]], dtype=float32)

In [24]:
#@Insight Plot the archive {display-mode: "form"}

# UMAP reduces the dimensions from 1024 to 2 dimensions that we can plot
reducer = umap.UMAP(n_neighbors=15)
umap_embeds = reducer.fit_transform(sentence_embeddings)
# Prepare the data to plot and interactive visualization
# using Altair
df_explore = pd.DataFrame(data={'text': df['Insight'], "InsightParent": df['InsightParent']})
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]
df_explore


,text,InsightParent,x,y
0,"Les intérimaires manquent de rigueur en termes de ponctualité, de professionnalisme, et d'attention au détail, ce qui se traduit par des erreurs coûteuses et un comportement inapproprié au travail.","Les intérimaires présentent des lacunes en termes de discipline au travail, impactant la qualité de leur performance et leur conformité aux processus.",1.802759,4.983328
1,"Les candidats indiquent ne pas être informés concernant l'entreprise, son activité et les détails des postes avant l'entretien, créant un manque d'adéquation et de préparation.","Les clients sont insatisfaits de la sélection des intérimaires, en lien avec les candidats se sentant mal informés sur les postes avant l'entretien, menant à une inadéquation profil/poste.",2.785907,5.320036
2,Le client suggère la mise en place de contrôles de références pour les nouveaux intérimaires n'ayant jamais travaillé avec l'agence.,Le client suggère la mise en place de contrôles de références pour les nouveaux intérimaires n'ayant jamais travaillé avec l'agence.,4.302900,3.668696
3,"Les intérimaires présentent des lacunes en termes de discipline au travail, impactant la qualité de leur performance et leur conformité aux processus.","Les intérimaires présentent des lacunes en termes de discipline au travail, impactant la qualité de leur performance et leur conformité aux processus.",2.042334,4.883059
4,Il est nécessaire d'améliorer l'exactitude des déclarations SI/SIR en fonction des expositions et risques associés à chaque poste de travail pour une meilleure traçabilité et un suivi efficace de la santé des intérimaires tout au long et après leur carrière.,Il est nécessaire d'améliorer l'exactitude des déclarations SI/SIR en fonction des expositions et risques associés à chaque poste de travail pour une meilleure traçabilité et un suivi efficace de la santé des intérimaires tout au long et après leur carrière.,2.315294,4.747339
...,...,...,...,...
292,Le client demande une augmentation de la réactivité de la part de l'entreprise.,Les clients sont insatisfaits à cause du manque de communication proactive et de la réactivité insuffisante de l'entreprise.,5.661032,4.575464
293,Il est suggéré de mettre en place un service disponible 24 heures sur 24.,"Les clients désirent des interactions personnelles plus directes et accessibles, avec une communication simplifiée grâce à un point de contact dédié et des services continuellement disponibles.",6.638662,4.600783
294,Les clients sont très satisfaits de la qualité des intérimaires proposés par Randstad.,Les clients expriment une satisfaction globale élevée quant à la qualité et au professionnalisme des talents fournis par Randstad.,8.145966,7.873084
295,"Le client a exprimé une satisfaction générale en déclarant que son expérience s'est ""très bien passée"".","Les clients apprécient grandement le professionnalisme, la qualité d'écoute et l'excellence des interactions avec la conseillère de Randstad, reflétant un service client hautement satisfaisant.",10.120708,5.967834


In [26]:

# Plot
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    color='InsightParent',
    tooltip=['text', 'InsightParent']
).properties(
    width=700,
    height=400
)
chart.interactive()

alt.Chart(...)

In [13]:
df_explore

,text,x,y
0,"Les intérimaires manquent de rigueur en termes de ponctualité, de professionnalisme, et d'attention au détail, ce qui se traduit par des erreurs coûteuses et un comportement inapproprié au travail.",2.039218,5.245661
1,"Les candidats indiquent ne pas être informés concernant l'entreprise, son activité et les détails des postes avant l'entretien, créant un manque d'adéquation et de préparation.",2.849567,5.300474
2,Le client suggère la mise en place de contrôles de références pour les nouveaux intérimaires n'ayant jamais travaillé avec l'agence.,4.028140,3.703257
3,"Les intérimaires présentent des lacunes en termes de discipline au travail, impactant la qualité de leur performance et leur conformité aux processus.",1.935771,5.254660
4,Il est nécessaire d'améliorer l'exactitude des déclarations SI/SIR en fonction des expositions et risques associés à chaque poste de travail pour une meilleure traçabilité et un suivi efficace de la santé des intérimaires tout au long et après leur carrière.,2.318301,4.898256
...,...,...,...
292,Le client demande une augmentation de la réactivité de la part de l'entreprise.,5.639562,5.114185
293,Il est suggéré de mettre en place un service disponible 24 heures sur 24.,5.426791,6.545606
294,Les clients sont très satisfaits de la qualité des intérimaires proposés par Randstad.,8.628681,7.235402
295,"Le client a exprimé une satisfaction générale en déclarant que son expérience s'est ""très bien passée"".",9.227494,4.499735


In [14]:
df_explore["Insight"] = 1

In [15]:
other_sentences = ["Les gens sont mécontents", "Les gens sont très contents", "Le produit est parfait", "Le produit doit être amélioré", "délais", "réactivité", "incohérence"]
other_sentence_embeddings = model.encode(other_sentences)

In [16]:
# Prepare the data to plot and interactive visualization
# using Altair


In [17]:
other_umap_embeds = reducer.transform(other_sentence_embeddings)

other_df = pd.DataFrame({"text":other_sentences, 'x': other_umap_embeds[:,0], 'y': other_umap_embeds[:,1], "Insight": 0})
other_df

,text,x,y,Insight
0,Les gens sont mécontents,3.597150,7.704711,0
1,Les gens sont très contents,8.921825,3.810601,0
2,Le produit est parfait,8.972766,4.612510,0
3,Le produit doit être amélioré,5.467335,5.774810,0
4,délais,1.411596,5.369362,0
5,réactivité,7.428763,6.444159,0
6,incohérence,2.167230,6.992319,0


In [18]:
df_explore_new = pd.concat([df_explore, other_df])

In [19]:
# Plot
chart = alt.Chart(df_explore_new).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['text'],
    color='Insight',
).properties(
    width=700,
    height=400
)
chart.interactive()

alt.Chart(...)

In [20]:
df_explore_new.head()

,text,x,y,Insight
0,"Les intérimaires manquent de rigueur en termes de ponctualité, de professionnalisme, et d'attention au détail, ce qui se traduit par des erreurs coûteuses et un comportement inapproprié au travail.",2.039218,5.245661,1
1,"Les candidats indiquent ne pas être informés concernant l'entreprise, son activité et les détails des postes avant l'entretien, créant un manque d'adéquation et de préparation.",2.849567,5.300474,1
2,Le client suggère la mise en place de contrôles de références pour les nouveaux intérimaires n'ayant jamais travaillé avec l'agence.,4.028140,3.703257,1
3,"Les intérimaires présentent des lacunes en termes de discipline au travail, impactant la qualité de leur performance et leur conformité aux processus.",1.935771,5.254660,1
4,Il est nécessaire d'améliorer l'exactitude des déclarations SI/SIR en fonction des expositions et risques associés à chaque poste de travail pour une meilleure traçabilité et un suivi efficace de la santé des intérimaires tout au long et après leur carrière.,2.318301,4.898256,1


In [21]:
df_explore_new['text'].to_list()

["Les intérimaires manquent de rigueur en termes de ponctualité, de professionnalisme, et d'attention au détail, ce qui se traduit par des erreurs coûteuses et un comportement inapproprié au travail.",
 "Les candidats indiquent ne pas être informés concernant l'entreprise, son activité et les détails des postes avant l'entretien, créant un manque d'adéquation et de préparation.",
 "Le client suggère la mise en place de contrôles de références pour les nouveaux intérimaires n'ayant jamais travaillé avec l'agence.",
 'Les intérimaires présentent des lacunes en termes de discipline au travail, impactant la qualité de leur performance et leur conformité aux processus.',
 "Il est nécessaire d'améliorer l'exactitude des déclarations SI/SIR en fonction des expositions et risques associés à chaque poste de travail pour une meilleure traçabilité et un suivi efficace de la santé des intérimaires tout au long et après leur carrière.",
 "Le client apprécie le professionnalisme durant sa visite à V

In [22]:
df_explore_new['embedding'] = reducer.transform(df_explore_new['text'])

ValueError: could not convert string to float: "Les intérimaires manquent de rigueur en termes de ponctualité, de professionnalisme, et d'attention au détail, ce qui se traduit par des erreurs coûteuses et un comportement inapproprié au travail."

In [ ]:
umap_embeds

array([[ 2.7892613 ,  4.7492185 ],
       [ 5.859206  ,  1.0253093 ],
       [ 5.5827217 ,  1.1252486 ],
       [ 2.3357992 ,  2.0950499 ],
       [-0.29825407,  5.843427  ],
       [ 4.9516044 ,  1.7748865 ],
       [ 1.1163592 ,  5.597881  ]], dtype=float32)